In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import json
import ast
import requests
import logging
import pandas as pd
import numpy as np
import awswrangler as wr
from matplotlib import pyplot as plt
import seaborn as sns

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url, return_service_token

In [26]:
def convert_mail_string_to_list(x):
    # print(f"Start test = {x}, {type(x)}")
    x = ast.literal_eval(x)
    # print(f"   test = {x}, type x: {type(x)}, type x[0]: {x[0]}, {type(x[0])}")
    if type(x) == str:
        x = ast.literal_eval(x)
        # print(f"      x = {x}, type x: {type(x)}")
    return x

def remove_none_emails(data, column_name):
    print(f"remove_none_emails {column_name} = {data.shape}")
    data = data[
        (data[column_name] != "None")
        & (data[column_name].isnull() == False)
        #& (data[column_name] != np.nan)
        #& (data[column_name] != None)
    ]
    print(f"remove_none_emails 2 {column_name} = {data.shape}")
    return data

def prepare_email_data(data, column_name):
    print(f"prepare_email_data 1 {column_name} = {data.shape}")
    data = remove_none_emails(data, column_name)
    print(f"prepare_email_data = {data.shape}")
    data[column_name] = (
        data[column_name]
        .astype("string")
        .apply(lambda x: convert_mail_string_to_list(x))
    )
    print(f"prepare_email_data 2 = {data.shape}")
    return data

def return_email_content(col_name: str, data: pd.DataFrame, is_value_based=False):
    if is_value_based:
        return [
            {
                "email": row[col_name][0]["id"],
                "value": row["value_based_conversion_value"], #config.col_value_based_conversion],
            }
            for _, row in data.iterrows()
        ]
    else:
        return [{"email": row[col_name][0]["id"]} for _, row in data.iterrows()]

def create_content_for_mail(data: pd.DataFrame, col_name: str):
    # TODO: refactor for other functions
    if col_name == "email":
        col_mail = "email_externalIds"
    elif col_name == "email_sha256":
        col_mail = "email_sha256_externalIds"
    logging.info(f"send_users = {data.shape}")
    profiles_mail_exists = prepare_email_data(
        data,
        col_mail,
    )
    if col_mail == "email_sha256_externalIds":
        print("hased")
        result_with_mail = return_email_hashed_content(
            col_mail, profiles_mail_exists
        )
    elif col_mail == "email_externalIds":
        print("normal")
        result_with_mail = return_email_content(col_mail, profiles_mail_exists)
    else:
        logging.warning(f"Unknown column name {col_name}")
        result_with_mail = []

    logging.info(f"result_with_mail = {len(result_with_mail)}")
    # combine
    profiles = []
    profiles.extend(result_with_mail)
    return profiles
    
# def create_content_for_mail(data: pd.DataFrame):
#     # TODO: refactor for other functions
#     col_email_hash = "email_sha256_externalIds"
#     col_email = "email_externalIds"
#     #print(f"send_users = {data.columns}")
#     profiles_mail_exists = prepare_email_data(
#         data,
#         col_email,
#     )
#     #print(f"profiles_mail_exists = {profiles_mail_exists}")
#     profiles_hashed_mails = prepare_email_data(
#         data,
#         col_email_hash,
#     )
#     #print(f"profiles_hased_mail_exists = {profiles_mail_exists}")
#     result_with_mail = return_email_content(col_email, profiles_mail_exists)
#     result_without_mail = return_email_hashed_content(
#         col_email_hash, profiles_hashed_mails
#     )

#     #print(f"profiles_hashed_mails = {len(profiles_hashed_mails)}")
#     # combine
#     profiles = []
#     profiles.extend(result_with_mail)
#     profiles.extend(result_without_mail)
#     return profiles

def return_email_hashed_content(
    col_name: str, data: pd.DataFrame, is_value_based=False
):
    if is_value_based:
        return [
            {
                "traits": {"email_sha256": row[col_name][0]["id"]},
                "value": row["value_based_conversion_value"],
            }
            for _, row in data.iterrows()
        ]

    else:
        return [
            {"traits": {"email_sha256": row[col_name][0]["id"]}}
            for _, row in data.iterrows()]

def create_content_for_cookieid(data: pd.DataFrame, cookie: str):
    # TODO: refactor for other functions
    col_external_id = f"{cookie}_externalIds"#{config.col_external_id}"
    logging.info(f"create_content_for_cookieid col_external_id = {col_external_id}")
    send_users = data.rename(columns={col_external_id: "externalIds"})
    send_users = send_users.dropna(subset=["externalIds"]).reset_index(drop=True)
    logging.info(f"create_content_for_cookieid = {send_users.columns}")
    profiles = (
        send_users.groupby("id")
        .agg({"anonymousId": "first", "externalIds": "first"})
        .reset_index()
        .to_dict("records")
    )
    return profiles


In [4]:
audience = "685e567a34f2e2097f339192"
customer = "Asambeauty"
date = "20250828"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [5]:
targeting = wr.s3.read_parquet(f"s3://{account_id}/targeting.history/{date}/{audience}.parquet")
targeting = targeting[targeting["treatment"]==audience]
print(len(targeting))
targeting.head()

In [27]:
send_users = targeting.copy()
send_users = send_users.rename(columns={"profile": "id"})
targeting_content = []
list_cookies = ["email", "criteo_gum_id","email_sha256"]
for cookie in list_cookies:
        print(f"create content for cookie = {cookie}")
        if "email" in cookie:
            content = create_content_for_mail(send_users, cookie)
            print(f"content = {content}")
        else:
            content = create_content_for_cookieid(send_users, cookie)
        if len(content) == 0:
            logging.warning(f"No content for cookie {cookie}")
        print(f"content {cookie} = {len(content)}")
        targeting_content.append(content)
len(targeting_content)

In [28]:
93726+16677

In [29]:
targeting_content = [item for sublist in targeting_content for item in sublist]
print(f"targeting_content = {len(targeting_content)}")

In [55]:
# gum_ids = [entry for entry in targeting_content if "criteo_gum_id" in entry["externalIds"]]
# email = [entry for entry in targeting_content if "email" in entry["externalIds"] or "emal_sha256" in entry["externalIds"]]
# len(email), len(gum_ids)

In [54]:
test=targeting_content
new_targeting_content = []
for ientry, entry in enumerate(test):
    print(entry)
    if "traits" in entry.keys():
        new_targeting_content.append(entry)
        continue
    if type(entry["externalIds"]) == list:
        new_targeting_content.append(entry)
        continue
    entry["externalIds"] = ast.literal_eval(entry["externalIds"])
    print(entry)
    new_targeting_content.append(entry)
len(new_targeting_content), new_targeting_content

In [58]:
payload = json.dumps(
        {
            "content": {
                "serviceName": "profiles/target_v2",
                "audience": audience,
                "profiles": new_targeting_content[50000::],
            },
            "context": {"accountId": account_id},
        }
    )
service_token = return_service_token()
headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {service_token}",
    }

In [59]:
status = requests.request(
            "POST",
            f"https://targeting.innkeepr.ai/api/core/queue/store",
            headers=headers,
             data=payload,
        )
status.json()

In [ ]:
#0 - 50k: 68b17b01e51c01d98a8a6c2e
#50k - ...: 68b17b82e51c01d98a8a6c2f